OpenCV notebook - ©Arkaprabha Majumdar

In [4]:
import cv2
import numpy as np

# #1. Sorting Contours
This is of two kinds:
Eliminating smaller contours, and contour in specific orders

In [12]:
image=cv2.pyrDown(cv2.imread('/home/arkaprabham/Desktop/Deep_Learning_Udemy/img/quotes.jpg'))

black_bg=np.zeros(image.shape)

grayscale=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

edges=cv2.Canny(grayscale,252,255)

contours,hierarchy=cv2.findContours(edges,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

print(len(contours))

cv2.drawContours(black_bg,[x for x in contours if cv2.contourArea(x)>10],-1,(0,255,0),3)
cv2.imshow("Contours on black",black_bg)
cv2.waitKey(0)
cv2.destroyAllWindows()

51


In [16]:
def getArea(contours):
    all_area=[]
    for cnt in contours:
        all_area.append(cv2.contourArea(cnt))
    return all_area

sorted_contours=sorted(contours,key=cv2.contourArea,reverse=True)
print(getArea(sorted_contours))

for c in sorted_contours:
    if getArea([c])[0]>100:
        cv2.drawContours(image,c,-1,(255,0,0),3)
        cv2.waitKey(0)
        cv2.imshow("Contour by area",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[1847.0, 1169.0, 1125.0, 980.0, 849.0, 736.0, 476.5, 442.5, 427.5, 409.5, 297.0, 254.0, 208.0, 172.0, 153.0, 150.0, 148.0, 142.0, 131.0, 129.0, 127.0, 113.5, 109.0, 104.0, 89.5, 77.5, 57.0, 46.0, 40.0, 29.0, 22.0, 20.5, 17.5, 11.0, 10.0, 10.0, 7.5, 4.0, 4.0, 4.0, 4.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [24]:
getArea([sorted_contours[2]])[0]

1125.0

In [17]:
image=cv2.pyrDown(cv2.imread('/home/arkaprabham/Desktop/Deep_Learning_Udemy/img/quotes.jpg'))

grayscale=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

edges=cv2.Canny(grayscale,252,255)

contours,hierarchy=cv2.findContours(edges,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

#If we dont re import the image, the annotations get stored on the image
def x_coord(contour):
    #returns the x coordinate of the contour centroid
    if cv2.contourArea(contour)>10:
        M=cv2.moments(contour)
        return int(M['m10']/M['m00'])
    
def label_center(image,c,i):
    #places a red circle on the contour centroid
    M=cv2.moments(c)
    if M['m00']>0:
        cx=int(M['m10']/M['m00'])
        cy=int(M['m01']/M['m00'])
        cv2.circle(image,(cx,cy),10,(0,0,255),-1)
        return image

#Compute centroid and draw image
for (i,c) in enumerate(contours):
    orig=label_center(image,c,i)
    
#print(contours)
#Sort left to right
def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0
    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

cnt_L2R,boundingRect=sort_contours(contours)

for (i,c) in enumerate(list(cnt_L2R)):
    M=cv2.moments(c)
    if M['m00']>0:
        cx=int(M['m10']/M['m00'])
        cy=int(M['m01']/M['m00'])
        cv2.putText(image,str(i+1),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.imshow("L2R contour",image)
        key=cv2.waitKey(0)
        if key==27: #Esc key to stop
            break
        elif key==13:
            (x,y,w,h) = boundingRect[i]
            print(boundingRect[i])
            #Then we crop each contour and save them in separate files
            cropped=image[y:y+h,x:x+w]
            image_name="output_cropped_shape_"+str(i+1)+".jpg"
            cv2.imwrite(image_name,cropped)
cv2.destroyAllWindows()

To find the key number of any key on your keyboard, use a function like below to print it.

In [18]:
'''img=cv2.pyrDown(cv2.imread('/home/arkaprabham/Desktop/Deep_Learning_Udemy/img/bear.jpg'))
while(1):
    cv2.imshow('img',img)
    k = cv2.waitKey(33)
    if k==27:    # Esc key to stop
        break
    elif k==-1:  # normally -1 returned,so don't print it
        continue
    else:
        print(k) # else print its value
cv2.destroyAllWindows()'''

#Space=32
#Esc=27
#Enter=13
#backspace=8

"img=cv2.pyrDown(cv2.imread('/home/arkaprabham/Desktop/Deep_Learning_Udemy/img/bear.jpg'))\nwhile(1):\n    cv2.imshow('img',img)\n    k = cv2.waitKey(33)\n    if k==27:    # Esc key to stop\n        break\n    elif k==-1:  # normally -1 returned,so don't print it\n        continue\n    else:\n        print(k) # else print its value\ncv2.destroyAllWindows()"

Note: The moments function in openCV gives 10 values of spatial moments, 7 values of central moments and 7 values of central norm moments. All other values are zero.

# #2. Contour Approximation
This approximates a contour. It is particularly useful if your image has small gaps.

In [6]:
image=cv2.imread('/home/arkaprabham/Desktop/Deep_Learning_Udemy/img/animals-better.jpg')
orig_image=image.copy()

#cv2.imshow("Original image",orig_image)

grayscale=cv2.cvtColor(orig_image,cv2.COLOR_BGR2GRAY)
denoised_image=cv2.GaussianBlur(grayscale,(5,5),0)

edges=cv2.Canny(denoised_image,100,150)
cv2.imshow("Canny",edges)

cnt,hier=cv2.findContours(edges,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

for c in cnt:
    x,y,w,h= cv2.boundingRect(c)
    cv2.rectangle(orig_image,(x,y),(x+w,y+h),(0,0,255),1)
    cv2.imshow("Bounding Rectangle",orig_image)

black_bg=np.zeros(orig_image.shape)
    
for c in cnt:
    accuracy=0.03*cv2.arcLength(c,True)
    approx=cv2.approxPolyDP(c,accuracy,True)
    cv2.drawContours(black_bg,[approx],0,(0,255,0),1)
    cv2.imshow("Approx poly DP",black_bg)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Convex Hull

This is similar to contour approximation.

In [8]:
n=len(cnt)
contours=sorted(cnt, key=cv2.contourArea,reverse=False)[:n]

#iterate through and draw hull
for c in contours:
    hull=cv2.convexHull(c)
    cv2.drawContours(image,[hull],0,(0,0,255),1)
    cv2.imshow("Convex Hull",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# #3. Matching Shapes
This is a basic identification which can be further used to identify complex shapes.

In [12]:
#First we create a template contour to match

hexagon_input=cv2.imread("/home/arkaprabham/Desktop/Deep_Learning_Udemy/img/hexagon.jpg") #basic template
gray=cv2.cvtColor(hexagon_input,cv2.COLOR_BGR2GRAY)
gray=cv2.erode(gray,(5,5))
#cv2.imshow("dfdg",gray)
_,thresh=cv2.threshold(gray,150,255,cv2.THRESH_BINARY_INV)
template_cnt,hier=cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
sorted_cont=sorted(template_cnt,key=cv2.contourArea,reverse=True)
cv2.drawContours(hexagon_input,sorted_cont[0],-1,(0,0,255),3)
cv2.imshow("threshold hex",hexagon_input)
cv2.waitKey(0)
cv2.destroyAllWindows()

#Done

51


In [ ]:
image=cv2.pyrDown(cv2.imread("/home/arkaprabham/Desktop/Deep_Learning_Udemy/img/shape scenery.jpg"))
blurred=cv2.GaussianBlur(image,(3,3),0)
black_bg=np.zeros(image.shape)
canny=cv2.Canny(blurred,50,150)
cv2.imshow("ssg",canny)
contours=cv2.findContours(canny,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]
for c in [x for x in contours if cv2.contourArea(x)>100]:
    match=cv2.matchShapes(sorted_cont[0],c,1,0.0)
    if match < 0.5 :
        closest_contour.append(c)
for c in closest_contour:
    hull=cv2.convexHull(c)
    cv2.drawContours(image,[hull],-1,(248,255,9),3)
    cv2.imshow("Output_",image)
    k=cv2.waitKey(0)
    if k==27: #Esc key to stop
        break
    elif k==13:
        continue
cv2.destroyAllWindows()

# Project : Identify and name shapes

In [14]:
new_img=cv2.imread("/home/arkaprabham/Desktop/Deep_Learning_Udemy/img/3d-Shapes.jpg")[:532,:] #crop words out
copied=new_img.copy()
cv2.imshow("Project 2",new_img)
cv2.waitKey(0)

grayscale=cv2.cvtColor(new_img,cv2.COLOR_BGR2GRAY)
blurred=cv2.GaussianBlur(grayscale,(5,5),cv2.BORDER_DEFAULT)
canny=cv2.Canny(blurred,100,200)
cv2.imshow("Canny",canny)
cv2.waitKey(0)

contours=cv2.findContours(canny,2,1)[0]
black_bg=np.zeros(new_img.shape)
for c in contours:
    #to get approximate polygons
    approx=cv2.approxPolyDP(c,0.01*cv2.arcLength(c,True),True)
    print(len(approx))
    #print("next")
    if len(approx)==6:
        cv2.drawContours(black_bg,[c],0,(83,83,233),-1)
        M=cv2.moments(c)
        if M['m00']>0:
            cx=int(M['m10']/M['m00'])
            cy=int(M['m01']/M['m00'])
        cv2.putText(black_bg,"hexagon",(cx-50,cy),cv2.FONT_HERSHEY_SIMPLEX,.2,(255,255,255),1)

    elif len(approx)==5:
        cv2.drawContours(black_bg,[c],0,(233,163,83),-1)
        M=cv2.moments(c)
        if M['m00']>0:
            cx=int(M['m10']/M['m00'])
            cy=int(M['m01']/M['m00'])
        cv2.putText(black_bg,"pentagon",(cx-50,cy),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
        
    elif len(approx)==4:
        cv2.drawContours(black_bg,[c],0,(63,182,50),-1)
        M=cv2.moments(c)
        if M['m00']>0:
            cx=int(M['m10']/M['m00'])
            cy=int(M['m01']/M['m00'])
        cv2.putText(black_bg,"parallelogram",(cx-50,cy),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
        
    elif len(approx)>15:
        cv2.drawContours(black_bg,[c],0,(0,128,255),-1)
        M=cv2.moments(c)
        if M['m00']>0:
            cx=int(M['m10']/M['m00'])
            cy=int(M['m01']/M['m00'])
        cv2.putText(black_bg,"circle",(cx-50,cy),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
        
cv2.imshow("found shapes",black_bg)
cv2.waitKey(0)
    
cv2.destroyAllWindows()

1
1
7
5
16
14
6
10
11
1
2
10
13
6
8
6
12
15
16
2
2
4
2
10
2
12
8
5
14
12
1
3
4
14
12
2
2
2
15
5
6
8


The answer depends on the presence of other shapes, level of noise if any and invariance you want to provide for (e.g. rotation, scaling, etc).
These requirements will define not only the algorithm but also required pre-procesing stages to extract features.